In [2]:
# %%
from pathlib import Path

import matplotlib.pyplot as plt

from contrai.models.semantic_segmentation.coat.predict import predict


In [6]:
# %%
# Example: adjust these to your actual locations
model_path = Path("/home/irortiza/Documents/PUBLIC_GITHUB/ContrAI/contrai/models/semantic_segmentation/coat/weights/contrail/model.pth")
image_path = Path("/home/irortiza/Documents/PUBLIC_GITHUB/ContrAI/notebooks/images/goes16_ash_rgb/2020/12/06/1600/ash_rgb_0p02deg.png")

# Optional: where to save the overlay result
output_path = Path("/home/irortiza/Documents/PUBLIC_GITHUB/ContrAI/outputs/contrail_overlay.png")

# Basic safety checks
assert model_path.exists(), f"Model weights not found: {model_path}"
assert image_path.exists(), f"Image not found: {image_path}"


In [7]:
# %%
overlay, mask, image = predict(
    model_path=model_path,
    image_path=image_path,
    tile_h=256,
    tile_w=256,
    stride=128,
    threshold=0.1,
    output=output_path,  # or None to skip saving
    show=False,          # we'll handle visualization below
    log_level="INFO",
)


2025-11-11 18:01:58,630 | INFO | Using device: cuda
2025-11-11 18:01:58,631 | INFO | Loading model from /home/irortiza/Documents/PUBLIC_GITHUB/ContrAI/contrai/models/semantic_segmentation/coat/weights/contrail/model.pth
2025-11-11 18:01:59,047 | INFO | Running inference: tile_h=256 tile_w=256 stride=128 threshold=0.100


_IncompatibleKeys(missing_keys=[], unexpected_keys=['norm1.weight', 'norm1.bias', 'norm2.weight', 'norm2.bias', 'norm3.weight', 'norm3.bias', 'norm4.weight', 'norm4.bias', 'aggregate.weight', 'aggregate.bias', 'head.weight', 'head.bias'])


2025-11-11 18:02:07,232 | INFO | Creating overlay
2025-11-11 18:02:09,523 | INFO | Saved overlay to /home/irortiza/Documents/PUBLIC_GITHUB/ContrAI/outputs/contrail_overlay.png
